In [1]:
!cp -r /kaggle/input/text-to-sign-language/sign-vq-transformer /kaggle/working/


In [2]:
%cd /kaggle/working/sign-vq-transformer

/kaggle/working/sign-vq-transformer


In [3]:
!pip install "numpy<2" \
             lightning==2.1.4 \
             torchmetrics \
             matplotlib \
             wandb \
             fastdtw \
             sacrebleu==2.4.0 \
             tokenizers \
             jiwer \
             fasttext \
             opencv-python \
             mediapipe \
             pycpd \
             ffmpeg \
             lmdb \
             jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 11.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install diffusers accelerate

In [5]:
!ln -s /kaggle/input/sign-vq-transformer-data/data ./data

In [6]:
!ln -s /kaggle/input/sign-vq-transformer-data/backTranslation_PHIX_model /kaggle/working/sign-vq-transformer/models/backTranslation_PHIX_model

In [7]:
import wandb
from kaggle_secrets import UserSecretsClient

# Get your W&B API key from Kaggle secrets
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Log in to W&B non-interactively
wandb.login(key=wandb_api_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [8]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

class RefinerDataset(Dataset):
    """
    A simple PyTorch Dataset to load the (noisy, clean) pose pairs
    created in Part 2.
    """
    def __init__(self, data_path: Path):
        """
        Loads the pre-processed data.

        Args:
            data_path (Path): Path to the .pt file (e.g., 'refiner_train_data.pt')
                              This file should contain a tuple: (noisy_tensors, clean_tensors)
        """
        print(f"Loading data from {data_path}...")
        try:
            # Load the tuple (noisy_poses, clean_poses)
            self.noisy_poses, self.clean_poses = torch.load(data_path, map_location="cpu")
            print("Data loaded successfully.")
        except Exception as e:
            print(f"Error loading data from {data_path}: {e}")
            print("Please ensure Part 2 was completed successfully and the file exists.")
            raise e

        assert len(self.noisy_poses) == len(self.clean_poses), \
            "Mismatch between number of noisy and clean samples."

    def __len__(self):
        """Returns the total number of samples."""
        return len(self.clean_poses)

    def __getitem__(self, idx):
        """
        Returns one sample of (noisy_condition, clean_target)
        """
        return {
            "noisy": self.noisy_poses[idx],
            "clean": self.clean_poses[idx]
        }

print("RefinerDataset class defined.")

RefinerDataset class defined.


In [10]:
import torch
import pathlib
import sys
from pathlib import Path
from tqdm import tqdm
import warnings

# --- Import from your repo ---
from helpers import load_config, find_best_model
from model_vq import VQ_Transformer
from dataset_vq import CodebookDataModule

# --- Fix for UnpicklingError ---
# Add pathlib to PyTorch's safe globals
# This is crucial for loading your VQ_Transformer model
torch.serialization.add_safe_globals([pathlib.PosixPath])

# Suppress a specific UserWarning from PyTorch Lightning
warnings.filterwarnings(
    "ignore",
    "Applied 'strict=True' when loading.*",
    UserWarning
)

print("All modules imported successfully.")

# --- 1. Setup Environment ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

vq_model_dir = Path("./models/vq_models/phix_codebook")
if not vq_model_dir.exists():
    print(f"ERROR: VQ Model directory not found at {vq_model_dir}")
    print("Please ensure your trained VQ model is linked correctly.")
    # Stop execution if model isn't found
    # sys.exit(1)
else:
    print(f"Loading VQ model from: {vq_model_dir}")

# --- 2. Load VQ Config and Checkpoint Path ---
vq_config = load_config(vq_model_dir / "config.yaml")
vq_checkpoint_path = vq_model_dir / find_best_model(str(vq_model_dir))
print(f"Found VQ checkpoint: {vq_checkpoint_path}")

# --- 3. Load DataModule to get data info and loaders ---
# Use batch sizes from config for faster processing
train_bs = vq_config["data"]["train_batch_size"]
dev_bs = vq_config["data"]["test_batch_size"]
print(f"Using Train BS: {train_bs}, Dev BS: {dev_bs}")

vq_dataset_module = CodebookDataModule(
    vq_config["data"],
    cuda=str(device),
    save_path=vq_model_dir
)

print("Setting up train data...")
vq_dataset_module.setup("train")
print("Setting up dev data...")
vq_dataset_module.setup("dev")

train_loader = vq_dataset_module.get_dataloader("train")
dev_loader = vq_dataset_module.get_dataloader("dev")

# Get necessary params for model init
input_size = vq_dataset_module.train.input_size
fps = vq_dataset_module.train.fps
print(f"Data loaded. Input size: {input_size}, FPS: {fps}")

# --- 4. Load Trained VQ-Transformer Model ---
print("Initializing VQ_Transformer model...")
# We pass dataset=None because we don't need its internal plotting features
# We already got the input_size and fps from the data module
vq_model = VQ_Transformer(
    config=vq_config,
    train_batch_size=train_bs,
    dev_batch_size=dev_bs,
    dataset=None,
    input_size=input_size,
    model_dir=str(vq_model_dir),
    fps=fps,
    loggers={},
)

print("Loading state dict from checkpoint...")
vq_checkpoint = torch.load(vq_checkpoint_path, map_location=device)
vq_model.load_state_dict(vq_checkpoint["state_dict"], strict=True)
vq_model = vq_model.to(device).eval()
print("VQ Model loaded and set to eval mode.")

# --- 5. Define Processing Function ---
def process_batches(loader, vq_model, split_name):
    """
    Iterates through a dataloader, passes batches through the VQ-VAE,
    and returns the (noisy_reconstruction, clean_original) pairs.
    """
    all_clean_poses = []
    all_noisy_poses = []

    with torch.no_grad():
        for batch in tqdm(loader, desc=f"Processing {split_name} split"):
            # Move batch to GPU
            batch = {k: v.to(device) for k, v in batch.items() if hasattr(v, 'to')}

            # Get clean pose (the original input)
            clean_pose = batch["src"]

            # Get noisy pose (the VQ-VAE's reconstruction)
            # We call model_forward with decode_codebook=True to get the
            # pose reconstructed from the "hard" (quantized) indices.
            # Returns: encoder_output, quantized_input, pose, counter, codebook_pose
            _, _, _, _, noisy_pose = vq_model.model_forward(**batch, decode_codebook=True)

            # Append to lists (moving to CPU to save GPU RAM)
            all_clean_poses.append(clean_pose.cpu())
            all_noisy_poses.append(noisy_pose.cpu())

    # Concatenate all batches into single tensors
    return torch.cat(all_noisy_poses), torch.cat(all_clean_poses)

# --- 6. Execute and Save ---
try:
    print("\nStarting to process training data...")
    # Note: We save (noisy, clean) as this is the standard (input, target) format
    noisy_train, clean_train = process_batches(train_loader, vq_model, "train")

    train_save_path = Path("./refiner_train_data.pt")
    torch.save((noisy_train, clean_train), train_save_path)
    print(f"\nSaved training data to {train_save_path}")
    print(f"  Noisy shape: {noisy_train.shape}, Clean shape: {clean_train.shape}")

    print("\nStarting to process validation data...")
    noisy_dev, clean_dev = process_batches(dev_loader, vq_model, "dev")

    dev_save_path = Path("./refiner_dev_data.pt")
    torch.save((noisy_dev, clean_dev), dev_save_path)
    print(f"\nSaved validation data to {dev_save_path}")
    print(f"  Noisy shape: {noisy_dev.shape}, Clean shape: {clean_dev.shape}")

    print("\n✅ Part 2: Data Pre-processing complete!")

except Exception as e:
    print(f"\nAn error occurred during processing: {e}")
    print("Please check your model paths and configurations.")

All modules imported successfully.
Using device: cuda
Loading VQ model from: models/vq_models/phix_codebook
Found VQ checkpoint: models/vq_models/phix_codebook/model-epoch=399-val_loss=0.00-val_MSE=0.00.ckpt
Using Train BS: 128, Dev BS: 128
Setting up train data...


Loading data:: 100%|██████████| 7060/7060 [00:01<00:00, 6987.21it/s]


Loaded 100594 from train
Setting up dev data...


Loading data:: 100%|██████████| 515/515 [00:00<00:00, 10586.59it/s]


Loaded 6774 from dev
Data loaded. Input size: 534, FPS: 12
Initializing VQ_Transformer model...
Loading state dict from checkpoint...
VQ Model loaded and set to eval mode.

Starting to process training data...


Processing train split: 100%|██████████| 786/786 [00:21<00:00, 35.99it/s]



Saved training data to refiner_train_data.pt
  Noisy shape: torch.Size([100594, 4, 534]), Clean shape: torch.Size([100594, 4, 534])

Starting to process validation data...


Processing dev split: 100%|██████████| 53/53 [00:01<00:00, 38.00it/s]



Saved validation data to refiner_dev_data.pt
  Noisy shape: torch.Size([6774, 4, 534]), Clean shape: torch.Size([6774, 4, 534])

✅ Part 2: Data Pre-processing complete!


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch import Tensor
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# UPDATED SimplePoseUNet (No Timesteps)
# -----------------------------------------------------------------
class SimplePoseRefinerNet(nn.Module):
    """
    This is now a Denoising Autoencoder, not a U-Net.
    It takes the "noisy" VQ-VAE pose and refines it.
    It does NOT need or use timesteps.
    """
    def __init__(self, in_channels, out_channels, hidden_dim=512):
        super().__init__()

        # We only take ONE input now: the noisy pose
        self.conv1 = nn.Conv1d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim*2, kernel_size=3, padding=1)

        # Middle
        self.conv3 = nn.Conv1d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1)

        # Decoder
        self.conv4 = nn.Conv1d(hidden_dim*2, hidden_dim, kernel_size=3, padding=1)
        # Output layer predicts the refined pose
        self.conv5 = nn.Conv1d(hidden_dim, out_channels, kernel_size=3, padding=1)

        self.norm1 = nn.GroupNorm(8, hidden_dim)
        self.norm2 = nn.GroupNorm(8, hidden_dim*2)
        self.norm3 = nn.GroupNorm(8, hidden_dim*2)
        self.norm4 = nn.GroupNorm(8, hidden_dim)

        self.act = nn.SiLU()

    def forward(self, x):
        # x: [B, C, T] (This is the VQ-VAE pose)

        # Encoder
        h1 = self.act(self.norm1(self.conv1(x)))
        h2 = self.act(self.norm2(self.conv2(h1)))

        # Middle
        h3 = self.act(self.norm3(self.conv3(h2)))

        # Decoder with skip connections
        h4 = self.act(self.norm4(self.conv4(h3 + h2)))  # Skip from h2

        # Output the refined pose
        # We add the input 'x' as a final residual connection.
        # This forces the model to learn only the *difference*
        # between the noisy pose and the clean pose.
        out = self.conv5(h4 + h1) + x

        return out

# -----------------------------------------------------------------
# UPDATED DiffusionRefiner (Now a Denoising Autoencoder)
# -----------------------------------------------------------------
class DiffusionRefiner(L.LightningModule):
    """
    This module is now a Denoising Autoencoder (DAE).
    It takes the VQ-VAE pose as input and outputs a refined pose.
    """
    def __init__(self,
                 pose_dim: int,
                 window_size: int,
                 learning_rate: float = 1e-4):
        super().__init__()
        self.save_hyperparameters()

        self.pose_dim = pose_dim
        self.window_size = window_size
        self.learning_rate = learning_rate

        # Use our simple refiner network
        self.model = SimplePoseRefinerNet(
            in_channels=pose_dim,
            out_channels=pose_dim,
            hidden_dim=512  # Increased size
        )

    def _transpose_data(self, x: Tensor) -> Tensor:
        """Helper to swap [B, T, C] -> [B, C, T]"""
        return x.permute(0, 2, 1)

    def _transpose_output(self, x: Tensor) -> Tensor:
        """Helper to swap [B, C, T] -> [B, T, C]"""
        return x.permute(0, 2, 1)

    def training_step(self, batch, batch_idx):
        # The "clean" pose is our target
        clean_pose = batch["clean"]

        # The "noisy" pose from the VQ-VAE is our input
        noisy_input_pose = batch["noisy"]

        # --- NEW LOGIC ---
        # 1. Transpose input for the model
        noisy_input_t = self._transpose_data(noisy_input_pose)

        # 2. Get the model's prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose back
        refined_pose = self._transpose_output(refined_pose_t)

        # 4. Calculate loss: How far is our prediction from the *clean* pose?
        loss = F.l1_loss(refined_pose, clean_pose)

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    @torch.no_grad()
    def run_inference(self, noisy_condition: Tensor) -> Tensor:
        """
        Runs one-step refinement.
        Args:
            noisy_condition (Tensor): The VQ-VAE output, shape [B, T, C].
        Returns:
            Tensor: The refined pose, shape [B, T, C].
        """
        # 1. Transpose input
        noisy_input_t = self._transpose_data(noisy_condition)

        # 2. Get model prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose output and return
        return self._transpose_output(refined_pose_t)

    def validation_step(self, batch, batch_idx):
        clean_pose = batch["clean"]
        noisy_pose = batch["noisy"]

        # Run inference (one-step refinement)
        refined_pose = self.run_inference(noisy_pose)

        # Calculate val_loss (how far is refined from clean)
        val_loss = F.l1_loss(refined_pose, clean_pose)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True)

        # Calculate refinement metrics
        baseline_metric = F.l1_loss(noisy_pose, clean_pose)
        refined_metric = val_loss # This is the same as val_loss

        self.log("val_baseline_L1", baseline_metric, on_step=False, on_epoch=True)
        self.log("val_refined_L1", refined_metric, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate
        )
        return optimizer

print("Fixed DiffusionRefiner (now a DAE) class defined.")

Fixed DiffusionRefiner (now a DAE) class defined.


In [12]:
import torch
import lightning as L
from torch.utils.data import DataLoader
from pathlib import Path
import warnings

# --- Import from your repo ---
from helpers import load_config
from dataset_vq import CodebookDataModule

# --- ADD THIS IMPORT ---
from lightning.pytorch.loggers import WandbLogger 
# ------------------------

# Import ModelCheckpoint and EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

# --- 1. Setup ---
L.seed_everything(42)
torch.set_float32_matmul_precision('medium')

SAVE_PATH = Path("./models/diffusion_refiner/")
SAVE_PATH.mkdir(parents=True, exist_ok=True)
print(f"Models will be saved to: {SAVE_PATH}")

TRAIN_DATA_PATH = Path("./refiner_train_data.pt")
DEV_DATA_PATH = Path("./refiner_dev_data.pt")

if not TRAIN_DATA_PATH.exists() or not DEV_DATA_PATH.exists():
    print("ERROR: Refiner data not found. Please run Part 2 successfully.")
else:
    print("Found refiner data files.")

# --- 2. Get Data Parameters from VQ Config ---
print("Loading VQ config to get data dimensions...")
vq_config = load_config("./models/vq_models/phix_codebook/config.yaml")
window_size = vq_config["data"]["window_size"]

# We need to instantiate the VQ dataset module just to get the `input_size`
try:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)

        vq_data_mod = CodebookDataModule(
            vq_config["data"],
            cuda="cpu",
            save_path=Path(vq_config["save_path"]) / vq_config["name"]
        )
        vq_data_mod.setup("test")
        pose_dim = vq_data_mod.test.input_size
        print(f"Detected Pose Dim (C): {pose_dim}, Window Size (T): {window_size}")
except Exception as e:
    print(f"Error while loading VQ data to get dimensions: {e}")
    print("Please ensure your data and model paths are correct.")

# --- 3. Create DataLoaders ---
print("Creating DataLoaders for the refiner...")
train_dataset = RefinerDataset(TRAIN_DATA_PATH)
dev_dataset = RefinerDataset(DEV_DATA_PATH)

train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
dev_loader = DataLoader(
    dev_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)
print("DataLoaders created.")

# --- 4. Initialize Model and Trainer ---
print("Initializing DiffusionRefiner model...")

model = DiffusionRefiner(
    pose_dim=pose_dim,
    window_size=window_size,
    learning_rate=1e-4
)

# --- ADD THIS SECTION TO INITIALIZE W&B ---
print("Initializing W&B Logger...")
wandb_logger = WandbLogger(
    project="sign-pose-refinement",  # Or any project name you like
    name="dae-refiner-run-1",        # Or any run name you like
    save_dir=SAVE_PATH,
    log_model=True                  # This will save the best model checkpoint to W&B
)

# Log hyperparameters (optional, but good practice)
wandb_logger.log_hyperparams({
    "model_type": "DenoisingAutoencoder",
    "pose_dim": pose_dim,
    "window_size": window_size,
    "learning_rate": 1e-4,
    "batch_size": 128,
    "epochs": 100
})
# ---------------------------------------------

checkpoint_callback = ModelCheckpoint(
    dirpath=SAVE_PATH,
    monitor="val_refined_L1",
    mode="min",
    save_top_k=1,
    filename="best_refiner_epoch={epoch}-L1={val_refined_L1:.4f}"
)

early_stopping_callback = EarlyStopping(
    monitor="val_refined_L1",
    mode="min",
    patience=15
)

print("Initializing Trainer...")
trainer = L.Trainer(
    max_epochs=100,
    accelerator="auto",
    devices=1,
    callbacks=[checkpoint_callback, early_stopping_callback],
    default_root_dir=SAVE_PATH,
    log_every_n_steps=10,
    logger=wandb_logger  # <--- PASS THE LOGGER HERE
)

# --- 5. Start Training ---
print("Starting training... This will take a while.")
print("Monitor 'val_refined_L1' in the progress bar. Lower is better.")

trainer.fit(model, train_loader, dev_loader)

print("\n🎉 Training complete!")
print(f"Best model saved to: {checkpoint_callback.best_model_path}")
print("You are now ready for Part 4 (Final Evaluation).")

INFO: Seed set to 42


Models will be saved to: models/diffusion_refiner
Found refiner data files.
Loading VQ config to get data dimensions...


Loading data:: 100%|██████████| 641/641 [00:00<00:00, 12894.66it/s]


Loaded 7875 from test
Detected Pose Dim (C): 534, Window Size (T): 4
Creating DataLoaders for the refiner...
Loading data from refiner_train_data.pt...
Data loaded successfully.
Loading data from refiner_dev_data.pt...
Data loaded successfully.
DataLoaders created.
Initializing DiffusionRefiner model...
Initializing W&B Logger...


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.



🎉 Training complete!
Best model saved to: /kaggle/working/sign-vq-transformer/models/diffusion_refiner/best_refiner_epoch=epoch=97-L1=val_refined_L1=0.0111.ckpt
You are now ready for Part 4 (Final Evaluation).


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch.utils.data import Dataset, DataLoader
from diffusers import DDPMScheduler
from torch import Tensor
from tqdm import tqdm
import numpy as np
import pathlib
import sys
from pathlib import Path
import warnings
import string

# --- Import from your repo ---
from helpers import load_config, find_best_model
from model_vq import VQ_Transformer
from model_translation import Transformer
from dataset_vq import CodebookDataModule
from dataset_translation import TranslationDataModule # Needed for vocab
from constants import PAD_ID, EOS_ID, BOS_ID
from stitch import stitch_poses
from metrics import pose_error_align_mje, bleu, rouge, chrf, wer
from back_translation import make_back_translation_model, back_translate

# --- Fix for UnpicklingError ---
torch.serialization.add_safe_globals([pathlib.PosixPath])

# --- Suppress Warnings ---
warnings.filterwarnings(
    "ignore", "Applied 'strict=True' when loading.*", UserWarning
)
warnings.filterwarnings(
    "ignore", "Missing logger folder*", UserWarning
)
warnings.filterwarnings(
    "ignore", ".*is not empty. Printing A LOT of messages*", UserWarning
)

print("All modules imported.")
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch import Tensor
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# UPDATED SimplePoseUNet (No Timesteps)
# -----------------------------------------------------------------
class SimplePoseRefinerNet(nn.Module):
    """
    This is now a Denoising Autoencoder, not a U-Net.
    It takes the "noisy" VQ-VAE pose and refines it.
    It does NOT need or use timesteps.
    """
    def __init__(self, in_channels, out_channels, hidden_dim=512):
        super().__init__()

        # We only take ONE input now: the noisy pose
        self.conv1 = nn.Conv1d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim*2, kernel_size=3, padding=1)

        # Middle
        self.conv3 = nn.Conv1d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1)

        # Decoder
        self.conv4 = nn.Conv1d(hidden_dim*2, hidden_dim, kernel_size=3, padding=1)
        # Output layer predicts the refined pose
        self.conv5 = nn.Conv1d(hidden_dim, out_channels, kernel_size=3, padding=1)

        self.norm1 = nn.GroupNorm(8, hidden_dim)
        self.norm2 = nn.GroupNorm(8, hidden_dim*2)
        self.norm3 = nn.GroupNorm(8, hidden_dim*2)
        self.norm4 = nn.GroupNorm(8, hidden_dim)

        self.act = nn.SiLU()

    def forward(self, x):
        # x: [B, C, T] (This is the VQ-VAE pose)

        # Encoder
        h1 = self.act(self.norm1(self.conv1(x)))
        h2 = self.act(self.norm2(self.conv2(h1)))

        # Middle
        h3 = self.act(self.norm3(self.conv3(h2)))

        # Decoder with skip connections
        h4 = self.act(self.norm4(self.conv4(h3 + h2)))  # Skip from h2

        # Output the refined pose
        # We add the input 'x' as a final residual connection.
        # This forces the model to learn only the *difference*
        # between the noisy pose and the clean pose.
        out = self.conv5(h4 + h1) + x

        return out

# -----------------------------------------------------------------
# UPDATED DiffusionRefiner (Now a Denoising Autoencoder)
# -----------------------------------------------------------------
class DiffusionRefiner(L.LightningModule):
    """
    This module is now a Denoising Autoencoder (DAE).
    It takes the VQ-VAE pose as input and outputs a refined pose.
    """
    def __init__(self,
                 pose_dim: int,
                 window_size: int,
                 learning_rate: float = 1e-4):
        super().__init__()
        self.save_hyperparameters()

        self.pose_dim = pose_dim
        self.window_size = window_size
        self.learning_rate = learning_rate

        # Use our simple refiner network
        self.model = SimplePoseRefinerNet(
            in_channels=pose_dim,
            out_channels=pose_dim,
            hidden_dim=512  # Increased size
        )

    def _transpose_data(self, x: Tensor) -> Tensor:
        """Helper to swap [B, T, C] -> [B, C, T]"""
        return x.permute(0, 2, 1)

    def _transpose_output(self, x: Tensor) -> Tensor:
        """Helper to swap [B, C, T] -> [B, T, C]"""
        return x.permute(0, 2, 1)

    def training_step(self, batch, batch_idx):
        # The "clean" pose is our target
        clean_pose = batch["clean"]

        # The "noisy" pose from the VQ-VAE is our input
        noisy_input_pose = batch["noisy"]

        # --- NEW LOGIC ---
        # 1. Transpose input for the model
        noisy_input_t = self._transpose_data(noisy_input_pose)

        # 2. Get the model's prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose back
        refined_pose = self._transpose_output(refined_pose_t)

        # 4. Calculate loss: How far is our prediction from the *clean* pose?
        loss = F.l1_loss(refined_pose, clean_pose)

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    @torch.no_grad()
    def run_inference(self, noisy_condition: Tensor) -> Tensor:
        """
        Runs one-step refinement.
        Args:
            noisy_condition (Tensor): The VQ-VAE output, shape [B, T, C].
        Returns:
            Tensor: The refined pose, shape [B, T, C].
        """
        # 1. Transpose input
        noisy_input_t = self._transpose_data(noisy_condition)

        # 2. Get model prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose output and return
        return self._transpose_output(refined_pose_t)

    def validation_step(self, batch, batch_idx):
        clean_pose = batch["clean"]
        noisy_pose = batch["noisy"]

        # Run inference (one-step refinement)
        refined_pose = self.run_inference(noisy_pose)

        # Calculate val_loss (how far is refined from clean)
        val_loss = F.l1_loss(refined_pose, clean_pose)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True)

        # Calculate refinement metrics
        baseline_metric = F.l1_loss(noisy_pose, clean_pose)
        refined_metric = val_loss # This is the same as val_loss

        self.log("val_baseline_L1", baseline_metric, on_step=False, on_epoch=True)
        self.log("val_refined_L1", refined_metric, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate
        )
        return optimizer

print("Fixed DiffusionRefiner (now a DAE) class defined.")

# -----------------------------------------------------------------
# CELL 2: LOAD ALL MODELS
# -----------------------------------------------------------------

@torch.no_grad()
def load_all_models(device):
    """Loads all 4 models and necessary configs."""

    print("--- Loading VQ-VAE Model (phix_codebook) ---")
    vq_model_dir = Path("./models/vq_models/phix_codebook")
    vq_config = load_config(vq_model_dir / "config.yaml")
    vq_ckpt_path = vq_model_dir / find_best_model(str(vq_model_dir))

    # We need input_size and fps from the data
    vq_data_mod = CodebookDataModule(
        vq_config["data"], cuda=str(device), save_path=vq_model_dir
    )
    vq_data_mod.setup("test")
    pose_dim = vq_data_mod.test.input_size
    window_size = vq_config["data"]["window_size"]
    fps = vq_data_mod.test.fps
    print(f"Pose Dim: {pose_dim}, Window Size: {window_size}, FPS: {fps}")

    vq_model = VQ_Transformer(
        config=vq_config, train_batch_size=1, dev_batch_size=1, dataset=None,
        input_size=pose_dim, model_dir=str(vq_model_dir), fps=fps, loggers={},
    )
    vq_ckpt = torch.load(vq_ckpt_path, map_location=device)
    vq_model.load_state_dict(vq_ckpt["state_dict"], strict=True)
    vq_model = vq_model.to(device).eval()
    codebook_pose = vq_model.get_codebook_pose().to(device)
    print("VQ-VAE Model loaded.")

    print("\n--- Loading Translation Model (phix_translation) ---")
    trans_model_dir = Path("./models/translation_models/phix_translation")
    trans_config = load_config(trans_model_dir / "config.yaml")
    trans_ckpt_path = trans_model_dir / find_best_model(str(trans_model_dir))

    # Load text vocab
    text_vocab_path = trans_model_dir / "text_vocab.txt"
    with open(text_vocab_path, 'r', encoding='utf-8') as f:
        text_vocab_list = [line.strip() for line in f.readlines()]
    text_vocab = {word: i for i, word in enumerate(text_vocab_list)}

    trans_model = Transformer(
        config=trans_config, save_path=trans_model_dir, train_batch_size=1,
        dev_batch_size=1, src_vocab=text_vocab,
        output_size=codebook_pose.shape[0] + 4, # +4 special tokens
        fps=fps, ground_truth_text={}, codebook_pose=codebook_pose,
    )
    trans_ckpt = torch.load(trans_ckpt_path, map_location=device)
    trans_model.load_state_dict(trans_ckpt["state_dict"], strict=True)
    trans_model = trans_model.to(device).eval()
    print("Translation Model loaded.")

    print("\n--- Loading Diffusion Refiner Model ---")
    refiner_model_dir = Path("./models/diffusion_refiner")

    # ------------------- THE FIX IS HERE -------------------
    # The refiner checkpoint has a different name format (e.g., "best_refiner...")
    # and doesn't contain the word "model", so find_best_model fails.
    # We will find it manually by looking for the first .ckpt file.
    try:
        # Find all .ckpt files in the directory
        ckpt_files = list(refiner_model_dir.glob("*.ckpt"))
        if not ckpt_files:
            raise FileNotFoundError(f"No .ckpt files found in {refiner_model_dir}")

        # Use the first one found (ModelCheckpoint saved only the best one)
        refiner_ckpt_path = ckpt_files[0]
        print(f"Found refiner checkpoint: {refiner_ckpt_path}")

    except Exception as e:
        print(f"Error finding refiner checkpoint: {e}")
        print("Please ensure your diffusion refiner model trained and saved correctly.")
        raise e
    # ------------------- END FIX -------------------

    refiner_model = DiffusionRefiner.load_from_checkpoint(
        refiner_ckpt_path,
        map_location=device,  # Add map_location
        pose_dim=pose_dim,
        window_size=window_size
    )
    refiner_model = refiner_model.to(device).eval()
    print("Diffusion Refiner Model loaded.")

    print("\n--- Loading Back-Translation Model ---")
    bt_model_dir = "./models/backTranslation_PHIX_model"
    bt_model = make_back_translation_model(bt_model_dir)
    bt_model = bt_model.to(device).eval()
    print("Back-Translation Model loaded.")

    models = {
        "vq_model": vq_model,
        "trans_model": trans_model,
        "refiner_model": refiner_model,
        "bt_model": bt_model
    }
    configs = {
        "vq_config": vq_config,
        "trans_config": trans_config
    }
    vocabs = {
        "text_vocab": text_vocab
    }

    return models, configs, vocabs

# -----------------------------------------------------------------
# CELL 3: LOAD TEST DATA
# -----------------------------------------------------------------

def load_test_data(data_path="./data/test.pt", text_vocab=None):
    """Loads the test.pt file and prepares it for inference."""
    raw_data = torch.load(data_path, map_location="cpu")

    test_samples = []
    for name, data in raw_data.items():
        # Pre-process text just like in dataset_translation.py
        text = data["text"].lower()
        text = text.replace("-", " ")
        remove_chars = (
            string.punctuation.replace(".", "") + "„“…–’‘”‘‚´" + "0123456789€"
        )
        text = "".join(ch for ch in text if ch not in remove_chars).split()

        # Tokenize
        token_indices = [text_vocab.get(w, text_vocab["<unk>"]) for w in text]
        token_indices.append(EOS_ID) # Add End-of-Sequence token

        # Store the raw text for BLEU/WER
        gt_text = " ".join(text)

        # Store the ground-truth pose
        gt_pose = data["poses_3d"]

        test_samples.append({
            "name": name,
            "token_indices": token_indices,
            "gt_text": gt_text,
            "gt_pose": gt_pose
        })
    print(f"Loaded {len(test_samples)} test samples.")
    return test_samples

# -----------------------------------------------------------------
# CELL 4: DEFINE THE FULL INFERENCE PIPELINE
# -----------------------------------------------------------------

@torch.no_grad()
def run_full_pipeline(sample, models, configs, device):
    """Runs a single text sample through the entire VQ + Refiner pipeline."""

    # Get models and configs
    trans_model = models["trans_model"]
    vq_model = models["vq_model"]
    refiner_model = models["refiner_model"]
    trans_config = configs["trans_config"]
    vq_config = configs["vq_config"]

    # --- 1. Create Batch for Translation Model ---
    token_indices = sample["token_indices"]
    src = torch.tensor(token_indices, dtype=torch.long, device=device).unsqueeze(0) # Batch size 1
    src_length = torch.tensor([len(token_indices)], dtype=torch.long, device=device)
    src_mask = (src != PAD_ID).unsqueeze(-2).to(device)

    # --- 2. Run Translation Model (Text -> VQ Tokens) ---
    model_settings = trans_config["model"]["beam_setting"]
    vq_tokens = trans_model.greedy_decode(
        src=src,
        src_length=src_length,
        src_mask=src_mask,
        max_output_length=model_settings["max_output_length"],
    )

    # --- 3. Post-process VQ Tokens ---
    vq_tokens = vq_tokens.squeeze(0).cpu().numpy() # Get first item in batch

    # Find the first EOS token
    eos_index = (vq_tokens == EOS_ID).nonzero()
    if eos_index[0].size > 0:
        eos_index = eos_index[0][0]
        vq_tokens = vq_tokens[:eos_index]

    # Shift tokens (remove special tokens)
    vq_tokens = vq_tokens[vq_tokens >= 4] - 4

    if len(vq_tokens) == 0:
        # Model predicted an empty sequence
        return None, None

    # --- 4. Convert VQ Tokens to Poses (Baseline) ---
    # `codebook_pose` is already on device
    baseline_pose_segments = trans_model.codebook_pose[vq_tokens]

    # --- 5. Refine Poses (Diffusion) ---
    # `run_inference` handles batching, but here batch size is N_tokens
    refined_pose_segments = refiner_model.run_inference(baseline_pose_segments)

    # --- 6. Stitch Poses ---
    window_size = vq_config["data"]["window_size"]
    stitch_config = trans_config["stitch"]

    # Reshape for stitching: [N, window_size, joints, 3]
    baseline_pose_segments_rs = baseline_pose_segments.reshape(
        baseline_pose_segments.shape[0], window_size, -1, 3
    )
    refined_pose_segments_rs = refined_pose_segments.reshape(
        refined_pose_segments.shape[0], window_size, -1, 3
    )

    # Stitch (poses must be on CPU for stitching function)
    stitched_baseline_pose = stitch_poses(
        poses=baseline_pose_segments_rs.cpu(),
        stitch_config=stitch_config
    )
    stitched_refined_pose = stitch_poses(
        poses=refined_pose_segments_rs.cpu(),
        stitch_config=stitch_config
    )

    return stitched_baseline_pose, stitched_refined_pose


# -----------------------------------------------------------------
# CELL 5: MAIN EVALUATION
# -----------------------------------------------------------------

def run_evaluation():
    L.seed_everything(42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"--- Starting Evaluation on {device} ---")

    # 1. Load Models
    models, configs, vocabs = load_all_models(device)

    # 2. Load Test Data
    test_samples = load_test_data(text_vocab=vocabs["text_vocab"])

    # 3. Initialize Lists to Store Results
    all_gt_text = []
    all_gt_poses = []
    all_baseline_poses_bt = [] # For Back-Translation
    all_refined_poses_bt = []  # For Back-Translation
    all_baseline_poses_mpjpe = [] # For MPJPE
    all_refined_poses_mpjpe = []  # For MPJPE

    # 4. Run Full Pipeline Loop
    print("\n--- Running Full Inference Pipeline on Test Set ---")
    for sample in tqdm(test_samples, desc="Evaluating"):

        baseline_pose, refined_pose = run_full_pipeline(
            sample, models, configs, device
        )

        # Get ground truth
        gt_pose = sample["gt_pose"]
        gt_text = sample["gt_text"]

        if baseline_pose is None:
            # Model predicted empty sequence, skip this sample
            continue

        # Store poses for Back-Translation (original stitched versions)
        all_baseline_poses_bt.append(baseline_pose)
        all_refined_poses_bt.append(refined_pose)

        # Store poses for MPJPE (must be aligned)
        all_baseline_poses_mpjpe.append(baseline_pose)
        all_refined_poses_mpjpe.append(refined_pose)
        all_gt_poses.append(gt_pose)

        # Store text
        all_gt_text.append(gt_text)

    print("\n--- Pipeline Complete. Calculating Metrics... ---")

    # 5. Calculate Metrics

    # --- Metric 1: Pose Fidelity (MPJPE) ---
    # We use pose_error_align_mje which uses DTW to align sequences
    print("Calculating MPJPE (with DTW alignment)...")

    # HACK: Temporarily convert ground truth poses to the right format
    # (N, K, 3) -> (N, K*3) -> (N, K, 3)
    # This is to match the format from stitch_poses
    all_gt_poses_formatted = []
    for p in all_gt_poses:
        # Make sure it's a tensor
        if not isinstance(p, torch.Tensor):
             p = torch.tensor(p)
        p_flat = torch.flatten(p, start_dim=-2, end_dim=-1)
        p_rs = p_flat.reshape(p_flat.shape[0], -1, 3)
        all_gt_poses_formatted.append(p_rs)

    baseline_mpjpe = pose_error_align_mje(
        hyps=all_baseline_poses_mpjpe, gt_pose=all_gt_poses_formatted
    )
    refined_mpjpe = pose_error_align_mje(
        hyps=all_refined_poses_mpjpe, gt_pose=all_gt_poses_formatted
    )

    # --- Metric 2: Back-Translation Quality (BLEU/WER) ---
    print("Calculating Back-Translation Metrics (BLEU/WER)...")
    bt_model = models["bt_model"]

    baseline_text_hyp = back_translate(bt_model, all_baseline_poses_bt)
    refined_text_hyp = back_translate(bt_model, all_refined_poses_bt)

    # BLEU
    bleu_baseline = bleu(baseline_text_hyp, all_gt_text)
    bleu_refined = bleu(refined_text_hyp, all_gt_text)

    # WER
    wer_baseline = wer(baseline_text_hyp, all_gt_text)
    wer_refined = wer(refined_text_hyp, all_gt_text)

    # 6. Print Final Results Table
    print("\n--- 🏆 FINAL RESULTS 🏆 ---")
    print("Lower is better for MPJPE and WER.")
    print("Higher is better for BLEU.")
    print("-" * 50)
    print(f"Metric          | Baseline (VQ-VAE) | Refined (Diffusion)")
    print("-" * 50)
    print(f"MPJPE (DTW)     | {baseline_mpjpe:<17.4f} | {refined_mpjpe:<17.4f}")
    print(f"WER             | {wer_baseline:<17.4f} | {wer_refined:<17.4f}")
    print(f"BLEU-1          | {bleu_baseline['bleu1']:<17.4f} | {bleu_refined['bleu1']:<17.4f}")
    print(f"BLEU-2          | {bleu_baseline['bleu2']:<17.4f} | {bleu_refined['bleu2']:<17.4f}")
    print(f"BLEU-3          | {bleu_baseline['bleu3']:<17.4f} | {bleu_refined['bleu3']:<17.4f}")
    print(f"BLEU-4          | {bleu_baseline['bleu4']:<17.4f} | {bleu_refined['bleu4']:<17.4f}")
    print("-" * 50)

# --- Run the evaluation! ---
run_evaluation()

INFO: Seed set to 42


All modules imported.
Fixed DiffusionRefiner (now a DAE) class defined.
--- Starting Evaluation on cuda ---
--- Loading VQ-VAE Model (phix_codebook) ---


Loading data:: 100%|██████████| 641/641 [00:00<00:00, 7728.18it/s]


Loaded 7875 from test
Pose Dim: 534, Window Size: 4, FPS: 12


Decoding codebook: 100%|██████████| 32/32 [00:00<00:00, 181.20it/s]


...


--- Pipeline Complete. Calculating Metrics... ---
Calculating MPJPE (with DTW alignment)...


Aligning A to B:: 100%|██████████| 641/641 [00:37<00:00, 17.22it/s]


Calculating Back-Translation Metrics (BLEU/WER)...


Running Back Translation: 100%|██████████| 21/21 [00:03<00:00,  6.39it/s]



--- 🏆 FINAL RESULTS 🏆 ---
Lower is better for MPJPE and WER.
Higher is better for BLEU.
--------------------------------------------------
Metric          | Baseline (VQ-VAE) | Refined (Diffusion)
--------------------------------------------------
MPJPE (DTW)     | 0.0394            | 0.0391           
WER             | 97.0909           | 95.8862          
BLEU-1          | 25.9924           | 25.8783          
BLEU-2          | 14.6043           | 14.5032          
BLEU-3          | 10.2652           | 10.1766          
BLEU-4          | 8.0193            | 7.9180           
--------------------------------------------------


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch.utils.data import Dataset, DataLoader
from diffusers import DDPMScheduler
from torch import Tensor
from tqdm import tqdm
import numpy as np
import pathlib
import sys
from pathlib import Path
import warnings
import string
from deep_translator import GoogleTranslator

# --- Import from your repo ---
from helpers import load_config, find_best_model
from model_vq import VQ_Transformer
from model_translation import Transformer
from dataset_vq import CodebookDataModule
from dataset_translation import TranslationDataModule # Needed for vocab
from constants import PAD_ID, EOS_ID, BOS_ID
from stitch import stitch_poses
from plot import make_pose_video # <-- This is the function we need
from back_translation import make_back_translation_model, back_translate

# --- Fix for UnpicklingError ---
torch.serialization.add_safe_globals([pathlib.PosixPath])

# --- Suppress Warnings ---
warnings.filterwarnings(
    "ignore", "Applied 'strict=True' when loading.*", UserWarning
)
warnings.filterwarnings(
    "ignore", "Missing logger folder*", UserWarning
)
warnings.filterwarnings(
    "ignore", ".*is not empty. Printing A LOT of messages*", UserWarning
)

print("All modules imported for video generation.")

import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch import Tensor
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# UPDATED SimplePoseUNet (No Timesteps)
# -----------------------------------------------------------------
class SimplePoseRefinerNet(nn.Module):
    """
    This is now a Denoising Autoencoder, not a U-Net.
    It takes the "noisy" VQ-VAE pose and refines it.
    It does NOT need or use timesteps.
    """
    def __init__(self, in_channels, out_channels, hidden_dim=512):
        super().__init__()

        # We only take ONE input now: the noisy pose
        self.conv1 = nn.Conv1d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim*2, kernel_size=3, padding=1)

        # Middle
        self.conv3 = nn.Conv1d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1)

        # Decoder
        self.conv4 = nn.Conv1d(hidden_dim*2, hidden_dim, kernel_size=3, padding=1)
        # Output layer predicts the refined pose
        self.conv5 = nn.Conv1d(hidden_dim, out_channels, kernel_size=3, padding=1)

        self.norm1 = nn.GroupNorm(8, hidden_dim)
        self.norm2 = nn.GroupNorm(8, hidden_dim*2)
        self.norm3 = nn.GroupNorm(8, hidden_dim*2)
        self.norm4 = nn.GroupNorm(8, hidden_dim)

        self.act = nn.SiLU()

    def forward(self, x):
        # x: [B, C, T] (This is the VQ-VAE pose)

        # Encoder
        h1 = self.act(self.norm1(self.conv1(x)))
        h2 = self.act(self.norm2(self.conv2(h1)))

        # Middle
        h3 = self.act(self.norm3(self.conv3(h2)))

        # Decoder with skip connections
        h4 = self.act(self.norm4(self.conv4(h3 + h2)))  # Skip from h2

        # Output the refined pose
        # We add the input 'x' as a final residual connection.
        # This forces the model to learn only the *difference*
        # between the noisy pose and the clean pose.
        out = self.conv5(h4 + h1) + x

        return out

# -----------------------------------------------------------------
# UPDATED DiffusionRefiner (Now a Denoising Autoencoder)
# -----------------------------------------------------------------
class DiffusionRefiner(L.LightningModule):
    """
    This module is now a Denoising Autoencoder (DAE).
    It takes the VQ-VAE pose as input and outputs a refined pose.
    """
    def __init__(self,
                 pose_dim: int,
                 window_size: int,
                 learning_rate: float = 1e-4):
        super().__init__()
        self.save_hyperparameters()

        self.pose_dim = pose_dim
        self.window_size = window_size
        self.learning_rate = learning_rate

        # Use our simple refiner network
        self.model = SimplePoseRefinerNet(
            in_channels=pose_dim,
            out_channels=pose_dim,
            hidden_dim=512  # Increased size
        )

    def _transpose_data(self, x: Tensor) -> Tensor:
        """Helper to swap [B, T, C] -> [B, C, T]"""
        return x.permute(0, 2, 1)

    def _transpose_output(self, x: Tensor) -> Tensor:
        """Helper to swap [B, C, T] -> [B, T, C]"""
        return x.permute(0, 2, 1)

    def training_step(self, batch, batch_idx):
        # The "clean" pose is our target
        clean_pose = batch["clean"]

        # The "noisy" pose from the VQ-VAE is our input
        noisy_input_pose = batch["noisy"]

        # --- NEW LOGIC ---
        # 1. Transpose input for the model
        noisy_input_t = self._transpose_data(noisy_input_pose)

        # 2. Get the model's prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose back
        refined_pose = self._transpose_output(refined_pose_t)

        # 4. Calculate loss: How far is our prediction from the *clean* pose?
        loss = F.l1_loss(refined_pose, clean_pose)

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    @torch.no_grad()
    def run_inference(self, noisy_condition: Tensor) -> Tensor:
        """
        Runs one-step refinement.
        Args:
            noisy_condition (Tensor): The VQ-VAE output, shape [B, T, C].
        Returns:
            Tensor: The refined pose, shape [B, T, C].
        """
        # 1. Transpose input
        noisy_input_t = self._transpose_data(noisy_condition)

        # 2. Get model prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose output and return
        return self._transpose_output(refined_pose_t)

    def validation_step(self, batch, batch_idx):
        clean_pose = batch["clean"]
        noisy_pose = batch["noisy"]

        # Run inference (one-step refinement)
        refined_pose = self.run_inference(noisy_pose)

        # Calculate val_loss (how far is refined from clean)
        val_loss = F.l1_loss(refined_pose, clean_pose)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True)

        # Calculate refinement metrics
        baseline_metric = F.l1_loss(noisy_pose, clean_pose)
        refined_metric = val_loss # This is the same as val_loss

        self.log("val_baseline_L1", baseline_metric, on_step=False, on_epoch=True)
        self.log("val_refined_L1", refined_metric, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate
        )
        return optimizer

print("Fixed DiffusionRefiner (now a DAE) class defined.")

# -----------------------------------------------------------------
# CELL 2: LOAD ALL MODELS
# -----------------------------------------------------------------

@torch.no_grad()
def load_all_models_for_inference(device):
    """Loads all 4 models and necessary configs."""

    print("--- Loading VQ-VAE Model (phix_codebook) ---")
    vq_model_dir = Path("./models/vq_models/phix_codebook")
    vq_config = load_config(vq_model_dir / "config.yaml")
    vq_ckpt_path = vq_model_dir / find_best_model(str(vq_model_dir))

    # We need input_size and fps from the data
    vq_data_mod = CodebookDataModule(
        vq_config["data"], cuda=str(device), save_path=vq_model_dir
    )
    vq_data_mod.setup("test")
    pose_dim = vq_data_mod.test.input_size
    window_size = vq_config["data"]["window_size"]
    fps = vq_data_mod.test.fps
    print(f"Pose Dim: {pose_dim}, Window Size: {window_size}, FPS: {fps}")

    vq_model = VQ_Transformer(
        config=vq_config, train_batch_size=1, dev_batch_size=1, dataset=None,
        input_size=pose_dim, model_dir=str(vq_model_dir), fps=fps, loggers={},
    )
    vq_ckpt = torch.load(vq_ckpt_path, map_location=device)
    vq_model.load_state_dict(vq_ckpt["state_dict"], strict=True)
    vq_model = vq_model.to(device).eval()
    codebook_pose = vq_model.get_codebook_pose().to(device)
    print("VQ-VAE Model loaded.")

    print("\n--- Loading Translation Model (phix_translation) ---")
    trans_model_dir = Path("./models/translation_models/phix_translation")
    trans_config = load_config(trans_model_dir / "config.yaml")
    trans_ckpt_path = trans_model_dir / find_best_model(str(trans_model_dir))

    text_vocab_path = trans_model_dir / "text_vocab.txt"
    with open(text_vocab_path, 'r', encoding='utf-8') as f:
        text_vocab_list = [line.strip() for line in f.readlines()]
    text_vocab = {word: i for i, word in enumerate(text_vocab_list)}

    trans_model = Transformer(
        config=trans_config, save_path=trans_model_dir, train_batch_size=1,
        dev_batch_size=1, src_vocab=text_vocab,
        output_size=codebook_pose.shape[0] + 4,
        fps=fps, ground_truth_text={}, codebook_pose=codebook_pose,
    )
    trans_ckpt = torch.load(trans_ckpt_path, map_location=device)
    trans_model.load_state_dict(trans_ckpt["state_dict"], strict=True)
    trans_model = trans_model.to(device).eval()
    print("Translation Model loaded.")

    print("\n--- Loading Diffusion Refiner Model ---")
    refiner_model_dir = Path("./models/diffusion_refiner")
    try:
        ckpt_files = list(refiner_model_dir.glob("*.ckpt"))
        if not ckpt_files:
            raise FileNotFoundError(f"No .ckpt files found in {refiner_model_dir}")
        refiner_ckpt_path = ckpt_files[0]
        print(f"Found refiner checkpoint: {refiner_ckpt_path}")
    except Exception as e:
        print(f"Error finding refiner checkpoint: {e}")
        raise e

    refiner_model = DiffusionRefiner.load_from_checkpoint(
        refiner_ckpt_path,
        map_location=device,
        pose_dim=pose_dim,
        window_size=window_size
    )
    refiner_model = refiner_model.to(device).eval()
    print("Diffusion Refiner Model loaded.")

    models = {
        "vq_model": vq_model,
        "trans_model": trans_model,
        "refiner_model": refiner_model
    }
    configs = {
        "vq_config": vq_config,
        "trans_config": trans_config
    }
    vocabs = {
        "text_vocab": text_vocab
    }

    return models, configs, vocabs, fps

# -----------------------------------------------------------------
# CELL 3: THE PREDICTION FUNCTION
# -----------------------------------------------------------------

# -----------------------------------------------------------------
# CELL 3: THE PREDICTION FUNCTION (Updated)
# -----------------------------------------------------------------

@torch.no_grad()
def predict_and_compare(
    english_sentence: str,
    models,
    configs,
    vocabs,
    fps,
    device,
    output_prefix: str = "comparison"  # <-- Changed to prefix
):
    """
    Runs a single English sentence through the full pipeline
    (Translation -> VQ-VAE -> Refiner) and saves two
    separate videos: one baseline, one refined.
    """

    print(f"--- Starting generation for: '{english_sentence}' ---")

    # --- Get models ---
    trans_model = models["trans_model"]
    vq_model = models["vq_model"]
    refiner_model = models["refiner_model"]
    trans_config = configs["trans_config"]
    vq_config = configs["vq_config"]
    text_vocab = vocabs["text_vocab"]

    # --- Step 1: Translate English to German ---
    print(f"Translating to German...")
    translator = GoogleTranslator(source='en', target='de')
    german_sentence = translator.translate(english_sentence)
    print(f"German: '{german_sentence}'")

    # --- Step 2: Pre-process and Tokenize German Text ---
    text = german_sentence.lower()
    text = text.replace("-", " ")
    remove_chars = (
        string.punctuation.replace(".", "") + "„“…–’‘”‘‚´" + "0123456789€"
    )
    text = "".join(ch for ch in text if ch not in remove_chars).split()
    token_indices = [text_vocab.get(w, text_vocab["<unk>"]) for w in text]
    token_indices.append(EOS_ID) # Add End-of-Sequence token

    # --- Step 3: Create Batch for Translation Model ---
    src = torch.tensor(token_indices, dtype=torch.long, device=device).unsqueeze(0)
    src_length = torch.tensor([len(token_indices)], dtype=torch.long, device=device)
    src_mask = (src != PAD_ID).unsqueeze(-2).to(device)

    # --- Step 4: Run Translation Model (Text -> VQ Tokens) ---
    print("Running Translation Model (Text -> VQ Tokens)...")
    model_settings = trans_config["model"]["beam_setting"]
    vq_tokens = trans_model.greedy_decode(
        src=src,
        src_length=src_length,
        src_mask=src_mask,
        max_output_length=model_settings["max_output_length"],
    )

    # --- Step 5: Post-process VQ Tokens ---
    vq_tokens = vq_tokens.squeeze(0).cpu().numpy()
    eos_index = (vq_tokens == EOS_ID).nonzero()
    if eos_index[0].size > 0:
        eos_index = eos_index[0][0]
        vq_tokens = vq_tokens[:eos_index]
    vq_tokens = vq_tokens[vq_tokens >= 4] - 4

    if len(vq_tokens) == 0:
        print("Model predicted an empty sequence. No video will be generated.")
        return
    print(f"Generated {len(vq_tokens)} VQ tokens.")

    # --- Step 6: Convert VQ Tokens to Poses (Baseline) ---
    baseline_pose_segments = trans_model.codebook_pose[vq_tokens]

    # --- Step 7: Refine Poses (Diffusion) ---
    print("Running Diffusion Refiner...")
    refined_pose_segments = refiner_model.run_inference(baseline_pose_segments)

    # --- Step 8: Stitch Poses for Video ---
    print("Stitching poses...")
    window_size = vq_config["data"]["window_size"]
    stitch_config = trans_config["stitch"]

    baseline_pose_segments_rs = baseline_pose_segments.reshape(
        baseline_pose_segments.shape[0], window_size, -1, 3
    )
    refined_pose_segments_rs = refined_pose_segments.reshape(
        refined_pose_segments.shape[0], window_size, -1, 3
    )

    # --- Create the baseline pose (NO extra processing) ---
    stitched_baseline_pose = stitch_poses(
        poses=baseline_pose_segments_rs.cpu(),
        stitch_config=stitch_config
    )
    
    # --- Create the refined pose (to BE processed) ---
    stitched_refined_pose = stitch_poses(
        poses=refined_pose_segments_rs.cpu(),
        stitch_config=stitch_config
    )

    # --- Step 9: Apply Refinements ONLY to the 'Refined' Pose ---
    
    # Import the helper functions
    from stitch import apply_low_pass_filter, interpolate_pose

    # 1. Apply a smoothing filter to the refined pose
    stitched_refined_pose_processed = apply_low_pass_filter(
        stitched_refined_pose, cutoff_freq=2.0, fs=fps 
    )
    
    # 2. Interpolate the refined pose to double the frames
    new_length = stitched_refined_pose_processed.shape[0] * 2
    stitched_refined_pose_processed = interpolate_pose(
        stitched_refined_pose_processed, num_sample_pts=new_length
    )

    # --- Step 10: Generate TWO Separate Videos ---
    print("Generating separate videos...")
    video_path = Path("./video_outputs/")
    video_path.mkdir(parents=True, exist_ok=True)

    # --- VIDEO 1: BASELINE (Robotic) ---
    # Rendered with original frames at original FPS
    make_pose_video(
        poses=[stitched_baseline_pose], 
        names=["Baseline (VQ-VAE)"],
        video_name=f"{output_prefix}_01_before.mp4",
        save_dir=video_path,
        fps=fps, # Original FPS
        overwrite=True
    )

    # --- VIDEO 2: REFINED (Smooth) ---
    # Rendered with 2x frames at 2x FPS (same duration, but smoother)
    make_pose_video(
        poses=[stitched_refined_pose_processed],
        names=["Refined (DAE)"],
        video_name=f"{output_prefix}_02_after.mp4",
        save_dir=video_path,
        fps=fps * 2, # Doubled FPS for interpolated frames
        overwrite=True
    )

    print(f"\n✅ Success! Videos saved to: {video_path}")
    print(f"   - {output_prefix}_01_before.mp4 (Baseline)")
    print(f"   - {output_prefix}_02_after.mp4 (Refined)")

# -----------------------------------------------------------------
# CELL 4: RUN THE PREDICTION (Updated)
# -----------------------------------------------------------------

# --- Load all models first ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models, configs, vocabs, fps = load_all_models_for_inference(device)

# --- Now, generate a video ---
english_input = "Temperatures tonight will be between four and nine degrees."
output_prefix_1 = "comparison_tonight_temp" # <-- Use prefix

predict_and_compare(
    english_input,
    models,
    configs,
    vocabs,
    fps,
    device,
    output_prefix_1 # <-- Pass prefix
)

# --- Example 2 ---
english_input_2 = "Tomorrow it will be sunny with a light wind."
output_prefix_2 = "comparison_sunny_wind" # <-- Use prefix

predict_and_compare(
    english_input_2,
    models,
    configs,
    vocabs,
    fps,
    device,
    output_prefix_2 # <-- Pass prefix
)

All modules imported for video generation.
Fixed DiffusionRefiner (now a DAE) class defined.
--- Loading VQ-VAE Model (phix_codebook) ---


Loading data:: 100%|██████████| 641/641 [00:00<00:00, 11804.93it/s]


Loaded 7875 from test
Pose Dim: 534, Window Size: 4, FPS: 12


Decoding codebook: 100%|██████████| 32/32 [00:00<00:00, 185.80it/s]


VQ-VAE Model loaded.

--- Loading Translation Model (phix_translation) ---
Translation Model loaded.

--- Loading Diffusion Refiner Model ---
Found refiner checkpoint: models/diffusion_refiner/best_refiner_epoch=epoch=97-L1=val_refined_L1=0.0111.ckpt
Diffusion Refiner Model loaded.
--- Starting generation for: 'Temperatures tonight will be between four and nine degrees.' ---
Translating to German...
German: 'Die Temperaturen liegen heute Nacht zwischen vier und neun Grad.'
Running Translation Model (Text -> VQ Tokens)...
Generated 16 VQ tokens.
Running Diffusion Refiner...
Stitching poses...
Generating separate videos...


Plotting poses: 100%|██████████| 128/128 [00:33<00:00,  3.80it/s]



✅ Success! Videos saved to: video_outputs
   - comparison_tonight_temp_01_before.mp4 (Baseline)
   - comparison_tonight_temp_02_after.mp4 (Refined)
--- Starting generation for: 'Tomorrow it will be sunny with a light wind.' ---
Translating to German...
German: 'Morgen wird es sonnig mit leichtem Wind.'
Running Translation Model (Text -> VQ Tokens)...
Generated 8 VQ tokens.
Running Diffusion Refiner...
Stitching poses...
Generating separate videos...


Plotting poses: 100%|██████████| 64/64 [00:17<00:00,  3.72it/s]



✅ Success! Videos saved to: video_outputs
   - comparison_sunny_wind_01_before.mp4 (Baseline)
   - comparison_sunny_wind_02_after.mp4 (Refined)


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch.utils.data import Dataset, DataLoader
from diffusers import DDPMScheduler
from torch import Tensor
from tqdm import tqdm
import numpy as np
import pathlib
import sys
from pathlib import Path
import warnings
import string
from deep_translator import GoogleTranslator

# --- Import from your repo ---
from helpers import load_config, find_best_model
from model_vq import VQ_Transformer
from model_translation import Transformer
from dataset_vq import CodebookDataModule
from dataset_translation import TranslationDataModule # Needed for vocab
from constants import PAD_ID, EOS_ID, BOS_ID
from stitch import stitch_poses
from plot import make_pose_video # <-- This is the function we need
from back_translation import make_back_translation_model, back_translate

# --- Fix for UnpicklingError ---
torch.serialization.add_safe_globals([pathlib.PosixPath])

# --- Suppress Warnings ---
warnings.filterwarnings(
    "ignore", "Applied 'strict=True' when loading.*", UserWarning
)
warnings.filterwarnings(
    "ignore", "Missing logger folder*", UserWarning
)
warnings.filterwarnings(
    "ignore", ".*is not empty. Printing A LOT of messages*", UserWarning
)

print("All modules imported for video generation.")

import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch import Tensor
from tqdm.auto import tqdm

# -----------------------------------------------------------------
# UPDATED SimplePoseUNet (No Timesteps)
# -----------------------------------------------------------------
class SimplePoseRefinerNet(nn.Module):
    """
    This is now a Denoising Autoencoder, not a U-Net.
    It takes the "noisy" VQ-VAE pose and refines it.
    It does NOT need or use timesteps.
    """
    def __init__(self, in_channels, out_channels, hidden_dim=512):
        super().__init__()

        # We only take ONE input now: the noisy pose
        self.conv1 = nn.Conv1d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim*2, kernel_size=3, padding=1)

        # Middle
        self.conv3 = nn.Conv1d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1)

        # Decoder
        self.conv4 = nn.Conv1d(hidden_dim*2, hidden_dim, kernel_size=3, padding=1)
        # Output layer predicts the refined pose
        self.conv5 = nn.Conv1d(hidden_dim, out_channels, kernel_size=3, padding=1)

        self.norm1 = nn.GroupNorm(8, hidden_dim)
        self.norm2 = nn.GroupNorm(8, hidden_dim*2)
        self.norm3 = nn.GroupNorm(8, hidden_dim*2)
        self.norm4 = nn.GroupNorm(8, hidden_dim)

        self.act = nn.SiLU()

    def forward(self, x):
        # x: [B, C, T] (This is the VQ-VAE pose)

        # Encoder
        h1 = self.act(self.norm1(self.conv1(x)))
        h2 = self.act(self.norm2(self.conv2(h1)))

        # Middle
        h3 = self.act(self.norm3(self.conv3(h2)))

        # Decoder with skip connections
        h4 = self.act(self.norm4(self.conv4(h3 + h2)))  # Skip from h2

        # Output the refined pose
        # We add the input 'x' as a final residual connection.
        # This forces the model to learn only the *difference*
        # between the noisy pose and the clean pose.
        out = self.conv5(h4 + h1) + x

        return out

# -----------------------------------------------------------------
# UPDATED DiffusionRefiner (Now a Denoising Autoencoder)
# -----------------------------------------------------------------
class DiffusionRefiner(L.LightningModule):
    """
    This module is now a Denoising Autoencoder (DAE).
    It takes the VQ-VAE pose as input and outputs a refined pose.
    """
    def __init__(self,
                 pose_dim: int,
                 window_size: int,
                 learning_rate: float = 1e-4):
        super().__init__()
        self.save_hyperparameters()

        self.pose_dim = pose_dim
        self.window_size = window_size
        self.learning_rate = learning_rate

        # Use our simple refiner network
        self.model = SimplePoseRefinerNet(
            in_channels=pose_dim,
            out_channels=pose_dim,
            hidden_dim=512  # Increased size
        )

    def _transpose_data(self, x: Tensor) -> Tensor:
        """Helper to swap [B, T, C] -> [B, C, T]"""
        return x.permute(0, 2, 1)

    def _transpose_output(self, x: Tensor) -> Tensor:
        """Helper to swap [B, C, T] -> [B, T, C]"""
        return x.permute(0, 2, 1)

    def training_step(self, batch, batch_idx):
        # The "clean" pose is our target
        clean_pose = batch["clean"]

        # The "noisy" pose from the VQ-VAE is our input
        noisy_input_pose = batch["noisy"]

        # --- NEW LOGIC ---
        # 1. Transpose input for the model
        noisy_input_t = self._transpose_data(noisy_input_pose)

        # 2. Get the model's prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose back
        refined_pose = self._transpose_output(refined_pose_t)

        # 4. Calculate loss: How far is our prediction from the *clean* pose?
        loss = F.l1_loss(refined_pose, clean_pose)

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    @torch.no_grad()
    def run_inference(self, noisy_condition: Tensor) -> Tensor:
        """
        Runs one-step refinement.
        Args:
            noisy_condition (Tensor): The VQ-VAE output, shape [B, T, C].
        Returns:
            Tensor: The refined pose, shape [B, T, C].
        """
        # 1. Transpose input
        noisy_input_t = self._transpose_data(noisy_condition)

        # 2. Get model prediction
        refined_pose_t = self.model(noisy_input_t)

        # 3. Transpose output and return
        return self._transpose_output(refined_pose_t)

    def validation_step(self, batch, batch_idx):
        clean_pose = batch["clean"]
        noisy_pose = batch["noisy"]

        # Run inference (one-step refinement)
        refined_pose = self.run_inference(noisy_pose)

        # Calculate val_loss (how far is refined from clean)
        val_loss = F.l1_loss(refined_pose, clean_pose)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True)

        # Calculate refinement metrics
        baseline_metric = F.l1_loss(noisy_pose, clean_pose)
        refined_metric = val_loss # This is the same as val_loss

        self.log("val_baseline_L1", baseline_metric, on_step=False, on_epoch=True)
        self.log("val_refined_L1", refined_metric, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate
        )
        return optimizer

print("Fixed DiffusionRefiner (now a DAE) class defined.")

# -----------------------------------------------------------------
# CELL 2: LOAD ALL MODELS
# -----------------------------------------------------------------

@torch.no_grad()
def load_all_models_for_inference(device):
    """Loads all 4 models and necessary configs."""

    print("--- Loading VQ-VAE Model (phix_codebook) ---")
    vq_model_dir = Path("./models/vq_models/phix_codebook")
    vq_config = load_config(vq_model_dir / "config.yaml")
    vq_ckpt_path = vq_model_dir / find_best_model(str(vq_model_dir))

    # We need input_size and fps from the data
    vq_data_mod = CodebookDataModule(
        vq_config["data"], cuda=str(device), save_path=vq_model_dir
    )
    vq_data_mod.setup("test")
    pose_dim = vq_data_mod.test.input_size
    window_size = vq_config["data"]["window_size"]
    fps = vq_data_mod.test.fps
    print(f"Pose Dim: {pose_dim}, Window Size: {window_size}, FPS: {fps}")

    vq_model = VQ_Transformer(
        config=vq_config, train_batch_size=1, dev_batch_size=1, dataset=None,
        input_size=pose_dim, model_dir=str(vq_model_dir), fps=fps, loggers={},
    )
    vq_ckpt = torch.load(vq_ckpt_path, map_location=device)
    vq_model.load_state_dict(vq_ckpt["state_dict"], strict=True)
    vq_model = vq_model.to(device).eval()
    codebook_pose = vq_model.get_codebook_pose().to(device)
    print("VQ-VAE Model loaded.")

    print("\n--- Loading Translation Model (phix_translation) ---")
    trans_model_dir = Path("./models/translation_models/phix_translation")
    trans_config = load_config(trans_model_dir / "config.yaml")
    trans_ckpt_path = trans_model_dir / find_best_model(str(trans_model_dir))

    text_vocab_path = trans_model_dir / "text_vocab.txt"
    with open(text_vocab_path, 'r', encoding='utf-8') as f:
        text_vocab_list = [line.strip() for line in f.readlines()]
    text_vocab = {word: i for i, word in enumerate(text_vocab_list)}

    trans_model = Transformer(
        config=trans_config, save_path=trans_model_dir, train_batch_size=1,
        dev_batch_size=1, src_vocab=text_vocab,
        output_size=codebook_pose.shape[0] + 4,
        fps=fps, ground_truth_text={}, codebook_pose=codebook_pose,
    )
    trans_ckpt = torch.load(trans_ckpt_path, map_location=device)
    trans_model.load_state_dict(trans_ckpt["state_dict"], strict=True)
    trans_model = trans_model.to(device).eval()
    print("Translation Model loaded.")

    print("\n--- Loading Diffusion Refiner Model ---")
    refiner_model_dir = Path("./models/diffusion_refiner")
    try:
        ckpt_files = list(refiner_model_dir.glob("*.ckpt"))
        if not ckpt_files:
            raise FileNotFoundError(f"No .ckpt files found in {refiner_model_dir}")
        refiner_ckpt_path = ckpt_files[0]
        print(f"Found refiner checkpoint: {refiner_ckpt_path}")
    except Exception as e:
        print(f"Error finding refiner checkpoint: {e}")
        raise e

    refiner_model = DiffusionRefiner.load_from_checkpoint(
        refiner_ckpt_path,
        map_location=device,
        pose_dim=pose_dim,
        window_size=window_size
    )
    refiner_model = refiner_model.to(device).eval()
    print("Diffusion Refiner Model loaded.")

    models = {
        "vq_model": vq_model,
        "trans_model": trans_model,
        "refiner_model": refiner_model
    }
    configs = {
        "vq_config": vq_config,
        "trans_config": trans_config
    }
    vocabs = {
        "text_vocab": text_vocab
    }

    return models, configs, vocabs, fps

# -----------------------------------------------------------------
# CELL 3: THE PREDICTION FUNCTION
# -----------------------------------------------------------------

# -----------------------------------------------------------------
# CELL 3: THE PREDICTION FUNCTION (Updated)
# -----------------------------------------------------------------

@torch.no_grad()
def predict_and_compare(
    english_sentence: str,
    models,
    configs,
    vocabs,
    fps,
    device,
    output_prefix: str = "comparison"  # <-- Changed to prefix
):
    """
    Runs a single English sentence through the full pipeline
    (Translation -> VQ-VAE -> Refiner) and saves two
    separate videos: one baseline, one refined.
    """

    print(f"--- Starting generation for: '{english_sentence}' ---")

    # --- Get models ---
    trans_model = models["trans_model"]
    vq_model = models["vq_model"]
    refiner_model = models["refiner_model"]
    trans_config = configs["trans_config"]
    vq_config = configs["vq_config"]
    text_vocab = vocabs["text_vocab"]

    # --- Step 1: Translate English to German ---
    print(f"Translating to German...")
    translator = GoogleTranslator(source='en', target='de')
    german_sentence = translator.translate(english_sentence)
    print(f"German: '{german_sentence}'")

    # --- Step 2: Pre-process and Tokenize German Text ---
    text = german_sentence.lower()
    text = text.replace("-", " ")
    remove_chars = (
        string.punctuation.replace(".", "") + "„“…–’‘”‘‚´" + "0123456789€"
    )
    text = "".join(ch for ch in text if ch not in remove_chars).split()
    token_indices = [text_vocab.get(w, text_vocab["<unk>"]) for w in text]
    token_indices.append(EOS_ID) # Add End-of-Sequence token

    # --- Step 3: Create Batch for Translation Model ---
    src = torch.tensor(token_indices, dtype=torch.long, device=device).unsqueeze(0)
    src_length = torch.tensor([len(token_indices)], dtype=torch.long, device=device)
    src_mask = (src != PAD_ID).unsqueeze(-2).to(device)

    # --- Step 4: Run Translation Model (Text -> VQ Tokens) ---
    print("Running Translation Model (Text -> VQ Tokens)...")
    model_settings = trans_config["model"]["beam_setting"]
    vq_tokens = trans_model.greedy_decode(
        src=src,
        src_length=src_length,
        src_mask=src_mask,
        max_output_length=model_settings["max_output_length"],
    )

    # --- Step 5: Post-process VQ Tokens ---
    vq_tokens = vq_tokens.squeeze(0).cpu().numpy()
    eos_index = (vq_tokens == EOS_ID).nonzero()
    if eos_index[0].size > 0:
        eos_index = eos_index[0][0]
        vq_tokens = vq_tokens[:eos_index]
    vq_tokens = vq_tokens[vq_tokens >= 4] - 4

    if len(vq_tokens) == 0:
        print("Model predicted an empty sequence. No video will be generated.")
        return
    print(f"Generated {len(vq_tokens)} VQ tokens.")

    # --- Step 6: Convert VQ Tokens to Poses (Baseline) ---
    baseline_pose_segments = trans_model.codebook_pose[vq_tokens]

    # --- Step 7: Refine Poses (Diffusion) ---
    print("Running Diffusion Refiner...")
    refined_pose_segments = refiner_model.run_inference(baseline_pose_segments)

    # --- Step 8: Stitch Poses for Video ---
    print("Stitching poses...")
    window_size = vq_config["data"]["window_size"]
    stitch_config = trans_config["stitch"]

    baseline_pose_segments_rs = baseline_pose_segments.reshape(
        baseline_pose_segments.shape[0], window_size, -1, 3
    )
    refined_pose_segments_rs = refined_pose_segments.reshape(
        refined_pose_segments.shape[0], window_size, -1, 3
    )

    # --- Create the baseline pose (NO extra processing) ---
    stitched_baseline_pose = stitch_poses(
        poses=baseline_pose_segments_rs.cpu(),
        stitch_config=stitch_config
    )
    
    # --- Create the refined pose (to BE processed) ---
    stitched_refined_pose = stitch_poses(
        poses=refined_pose_segments_rs.cpu(),
        stitch_config=stitch_config
    )

    # --- Step 9: Apply Refinements ONLY to the 'Refined' Pose ---
    
    # Import the helper functions
    from stitch import apply_low_pass_filter, interpolate_pose

    # 1. Apply a smoothing filter to the refined pose
    stitched_refined_pose_processed = apply_low_pass_filter(
        stitched_refined_pose, cutoff_freq=2.0, fs=fps 
    )
    
    # 2. Interpolate the refined pose to double the frames
    new_length = stitched_refined_pose_processed.shape[0] * 2
    stitched_refined_pose_processed = interpolate_pose(
        stitched_refined_pose_processed, num_sample_pts=new_length
    )

    # --- Step 10: Generate TWO Separate Videos ---
    print("Generating separate videos...")
    video_path = Path("./video_outputs/")
    video_path.mkdir(parents=True, exist_ok=True)

    # --- VIDEO 1: BASELINE (Robotic) ---
    # Rendered with original frames at original FPS
    make_pose_video(
        poses=[stitched_baseline_pose], 
        names=["Baseline (VQ-VAE)"],
        video_name=f"{output_prefix}_01_before.mp4",
        save_dir=video_path,
        fps=fps, # Original FPS
        overwrite=True
    )

    # --- VIDEO 2: REFINED (Smooth) ---
    # Rendered with 2x frames at 2x FPS (same duration, but smoother)
    make_pose_video(
        poses=[stitched_refined_pose_processed],
        names=["Refined (DAE)"],
        video_name=f"{output_prefix}_02_after.mp4",
        save_dir=video_path,
        fps=fps * 2, # Doubled FPS for interpolated frames
        overwrite=True
    )

    print(f"\n✅ Success! Videos saved to: {video_path}")
    print(f"   - {output_prefix}_01_before.mp4 (Baseline)")
    print(f"   - {output_prefix}_02_after.mp4 (Refined)")
    
# -----------------------------------------------------------------
# CELL 4: RUN THE PREDICTION (Updated)
# -----------------------------------------------------------------

# --- Load all models first ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models, configs, vocabs, fps = load_all_models_for_inference(device)

# --- Now, generate a video ---
english_input = "Temperatures tonight will be between four and nine degrees."
output_prefix_1 = "comparison_tonight_temp" # <-- Use prefix

predict_and_compare(
    english_input,
    models,
    configs,
    vocabs,
    fps,
    device,
    output_prefix_1 # <-- Pass prefix
)

# --- Example 2 ---
english_input_2 = "Tomorrow it will be sunny with a light wind."
output_prefix_2 = "comparison_sunny_wind" # <-- Use prefix

predict_and_compare(
    english_input_2,
    models,
    configs,
    vocabs,
    fps,
    device,
    output_prefix_2 # <-- Pass prefix
)

All modules imported for video generation.
Fixed DiffusionRefiner (now a DAE) class defined.
--- Loading VQ-VAE Model (phix_codebook) ---


Loading data:: 100%|██████████| 641/641 [00:00<00:00, 12656.46it/s]


Loaded 7875 from test
Pose Dim: 534, Window Size: 4, FPS: 12


Decoding codebook: 100%|██████████| 32/32 [00:00<00:00, 186.40it/s]


VQ-VAE Model loaded.

--- Loading Translation Model (phix_translation) ---
Translation Model loaded.

--- Loading Diffusion Refiner Model ---
Found refiner checkpoint: models/diffusion_refiner/best_refiner_epoch=epoch=97-L1=val_refined_L1=0.0111.ckpt
Diffusion Refiner Model loaded.
--- Starting generation for: 'Temperatures tonight will be between four and nine degrees.' ---
Translating to German...
German: 'Die Temperaturen liegen heute Nacht zwischen vier und neun Grad.'
Running Translation Model (Text -> VQ Tokens)...
Generated 16 VQ tokens.
Running Diffusion Refiner...
Stitching poses...
Generating separate videos...


...

Video already exists: video_outputs/comparison_tonight_temp_02_after.mp4.avi Overwritting

✅ Success! Videos saved to: video_outputs
   - comparison_tonight_temp_01_before.mp4 (Baseline)
   - comparison_tonight_temp_02_after.mp4 (Refined)
--- Starting generation for: 'Tomorrow it will be sunny with a light wind.' ---
Translating to German...
German: 'Morgen wird es sonnig mit leichtem Wind.'
Running Translation Model (Text -> VQ Tokens)...
Generated 8 VQ tokens.
Running Diffusion Refiner...
Stitching poses...
Generating separate videos...


Plotting poses: 100%|██████████| 32/32 [00:08<00:00,  3.74it/s]


Video already exists: video_outputs/comparison_sunny_wind_01_before.mp4.avi Overwritting


Plotting poses: 100%|██████████| 64/64 [00:17<00:00,  3.74it/s]


Video already exists: video_outputs/comparison_sunny_wind_02_after.mp4.avi Overwritting

✅ Success! Videos saved to: video_outputs
   - comparison_sunny_wind_01_before.mp4 (Baseline)
   - comparison_sunny_wind_02_after.mp4 (Refined)


In [17]:
import os
import subprocess

input_dir = "/kaggle/working/sign-vq-transformer/video_outputs"
output_dir = "/kaggle/working/sign-vq-transformer/converted_outputs"

# Create output folder if it doesn’t exist
os.makedirs(output_dir, exist_ok=True)

# Loop through all .mp4 and .avi files in input directory
for file in os.listdir(input_dir):
    if file.endswith((".mp4", ".avi")):
        input_path = os.path.join(input_dir, file)
        # remove existing extension and add .mp4
        base_name = os.path.splitext(file)[0]
        output_path = os.path.join(output_dir, base_name + ".mp4")

        print(f"Converting: {file} → {base_name}.mp4")

        # Run ffmpeg conversion (silent mode)
        subprocess.run([
            "ffmpeg", "-i", input_path,
            "-vcodec", "libx264", "-acodec", "aac",
            "-y", output_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

print("✅ Conversion complete! Check /content/converted_videos/")


Converting: comparison_tonight_temp_02_after.mp4.avi → comparison_tonight_temp_02_after.mp4.mp4
Converting: comparison_sunny_wind_01_before.mp4.avi → comparison_sunny_wind_01_before.mp4.mp4
Converting: comparison_sunny_wind_02_after.mp4.avi → comparison_sunny_wind_02_after.mp4.mp4
Converting: comparison_tonight_temp_01_before.mp4.avi → comparison_tonight_temp_01_before.mp4.mp4
✅ Conversion complete! Check /content/converted_videos/


In [18]:
import os
from IPython.display import Video, display

video_dir = "/kaggle/working/sign-vq-transformer/converted_outputs"

# List all .mp4 files in the directory
videos = [f for f in os.listdir(video_dir) if f.endswith(".mp4")]

# Display each video
for v in videos:
    print(f"🎬 {v}")
    display(Video(os.path.join(video_dir, v), embed=True, width=640, height=360))


🎬 comparison_sunny_wind_02_after.mp4.mp4


🎬 comparison_tonight_temp_02_after.mp4.mp4


🎬 comparison_sunny_wind_01_before.mp4.mp4


🎬 comparison_tonight_temp_01_before.mp4.mp4


In [20]:
from IPython.display import HTML
from base64 import b64encode

def show_videos_synced(video1_path, video2_path, width=400):
    # Read and encode videos
    video1 = open(video1_path, 'rb').read()
    video2 = open(video2_path, 'rb').read()
    video1_b64 = b64encode(video1).decode()
    video2_b64 = b64encode(video2).decode()
    
    html = f"""
    <div style="display:flex;justify-content:center;gap:10px;align-items:center;">
      <video id="vid1" width="{width}">
        <source src="data:video/mp4;base64,{video1_b64}" type="video/mp4">
      </video>
      <video id="vid2" width="{width}">
        <source src="data:video/mp4;base64,{video2_b64}" type="video/mp4">
      </video>
    </div>
    <div style="text-align:center;margin-top:10px;">
      <button onclick="playBoth()">▶ Play Both</button>
      <button onclick="pauseBoth()">⏸ Pause Both</button>
    </div>

    <script>
      function playBoth() {{
        const v1 = document.getElementById('vid1');
        const v2 = document.getElementById('vid2');
        v1.currentTime = 0;
        v2.currentTime = 0;
        Promise.all([v1.play(), v2.play()]);
      }}
      function pauseBoth() {{
        document.getElementById('vid1').pause();
        document.getElementById('vid2').pause();
      }}
    </script>
    """
    return HTML(html)

# Example usage
show_videos_synced(
    "/kaggle/working/sign-vq-transformer/converted_outputs/comparison_tonight_temp_01_before.mp4.mp4",
    "/kaggle/working/sign-vq-transformer/converted_outputs/comparison_tonight_temp_02_after.mp4.mp4"
)